In [414]:
import pandas as pd
import numpy as np
import datetime as dt
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc

In [415]:
bpl.output_notebook()

Loading BokehJS ...

In [416]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "05/19/2020"

In [417]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Adquisición de datos históricos, esta parte solo es necesaria una vez, guarda los archivos a disco

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [418]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [419]:
frames = []
for fecha in fechas_csv:
    data = pd.read_csv('./datos_federales/' + fecha,encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    frames.append(data)

In [420]:
len(frames)

38

In [421]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [422]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [423]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [424]:
confirmados = all_data[all_data["RESULTADO"]==1]

In [425]:
por_fecha = confirmados.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [426]:
fechas_validas = por_fecha.columns

In [427]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [428]:
conf_acum = por_fecha.sum().to_frame("Confirmados").reset_index()

In [429]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [430]:
p.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=1,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='153807', ...)

In [431]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i])

In [432]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [433]:
bpl.show(p)

In [434]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [435]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [436]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [437]:
bpl.show(s)

In [438]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [439]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())*100/(por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [440]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [441]:
bpl.show(s)

In [442]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    conf_sint.name = fecha
    df_fecha.append(conf_sint)

In [443]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-1]]

In [444]:
cambios = valores.pct_change()*100

In [445]:
fechas_casos = cambios.columns

In [446]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [447]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [448]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,10)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [449]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [450]:
bpl.show(s1)